# Machine Learning - Collaborative Filtering Algorithm for Movie Recommendations

The objective of this mini project is to build a collaborative filtering algorithm that able to make a proper movie recommendation based on user's rating on particular movies.

The data being used in this project was taken from MovieLens 100K Dataset, which contains a data of 1682 movies and their rating from 943 different users.

Before the implementation of collaborative filtering algorithm, it is necessary to do data wrangling first, in order to transform the data the way we want to observe it.

First, let's import the necessary libraries.

In [99]:
import numpy as np
import pandas as pd

Then read the data into a dataframe. To have a better sense of what the data looks like and to check whether it has been read properly, then it is essential to check it with head() function.

In [100]:
fullDataFrame = pd.read_csv('RawData.txt', delimiter="\t", names=["UserID","ItemID","Rating","Timestamp"])
fullDataFrame.head()

,UserID,ItemID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


From the data above, we can see the userID, the ItemID or movieID, the rating, and the timestamp. Out of these four columns, we don't actually need the Timestamp column for the investigation, so it is necessary to remove the Timestamp column.

In [101]:
dataFrame = fullDataFrame.drop(['Timestamp'], axis= 1)
dataFrame.head()

,UserID,ItemID,Rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


After that, we can fetch the total number of users as well as the movies.

In [102]:
totalUser = len(dataFrame.UserID.value_counts())
totalMovie = len(dataFrame.ItemID.value_counts())

We use the total number of users as well as total number of movies in order to build a matrix $Y$ which has a dimension of $m x n$, with $m$ being the total number of movies and $n$ the total number of users.

In [103]:
dataArray = dataFrame.to_numpy()

Y = np.zeros((totalMovie, totalUser))
for i in range (len(dataArray)):
        
        # -1 because Python indexing starts from 0.
        
        Y[(dataArray[i,1]-1), (dataArray[i,0]-1)] = dataArray[i,2]
Y[0:5,0:5]

array([[5., 4., 0., 0., 4.],
       [3., 0., 0., 0., 3.],
       [4., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0.],
       [3., 0., 0., 0., 0.]])

Here the 1682x943 $Y$ matrix has been built. The value 0 corresponds to the movie that the user hasn't rated yet. The rating system range between 1-5, with 1 being the lowest and 5 the highest.

After that, it is necessary to build $R$ matrix with the same dimension as matrix $Y$ above. The matrix $R$ contains two values only, 1 if a user already rated a movie or 0 otherwise.

In [104]:
# Initialize R matrix, which has a value of 1 if a user has rated a movie, or 0 otherwise.
R = np.zeros((totalMovie, totalUser))

for i in range (totalMovie):
    for j in range (totalUser):

        if Y[i,j] != 0:
            R[i,j] = 1
R[0:5,0:5]

array([[1., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

After that, let's import all the movie list provided by the dataset. In the dataset, there are 1682 movies in total. The movie list is provided by 'movie_ids.txt'.

In [105]:
def loadMovieList(strings):
    
    with open (strings,'r') as data:
        
        readData = data.readlines()
        
    readData = [x.strip() for x in readData]
    movieData = []
    
    for i in range (len(readData)):
        
        readData[i] = readData[i].split(' ',1)
        movieData.append(readData[i][1])
        
    return movieData

movieList = loadMovieList('movie_ids.txt')
movieList[0:5]

['Toy Story (1995)',
 'GoldenEye (1995)',
 'Four Rooms (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)']

Now we have successfully import the movie list. As you can see above, Toy Story has the first index in the movie list, followed by 007's GoldenEye. 

Before the collaborative filtering algorithm can do us a favor, let's rate several movies that we've seen already. Let's say I like chick flicks, comedy, and drama kind of movies and I don't like action or crime movies, then I would rate the movies at the beginning like the following.

In [106]:
# Initialize my movie rating.

myMovieRating = np.zeros((totalMovie,1))

# Movie I like
myMovieRating[169] = 5 #Cinema Paradiso
myMovieRating[215] = 5 #When Harry Met Sally
myMovieRating[245] = 4 #Chasing Amy
myMovieRating[250] = 5 #Shall We Dance?
myMovieRating[254] = 4 #My Best Friend's Wedding
myMovieRating[353] = 4 #The Wedding Singer
myMovieRating[87] = 5 #Sleepless in Seattle
myMovieRating[93] = 5 #Home Alone

# Movie I dislike
myMovieRating[352] = 2 #Deep Rising
myMovieRating[372] = 1 #Judge Dredd
myMovieRating[478] = 2 #Vertigo
myMovieRating[591] = 2 #True Crime
myMovieRating[602] = 2 #Rear Window
myMovieRating[690] = 1 #Dark City

Then, print the rating and movie titles for validation.

In [107]:
for i in range (len(myMovieRating)):
    
    if myMovieRating[i] != 0:
        print('I rated', movieList[i], int(myMovieRating[i]),' stars')

I rated Sleepless in Seattle (1993) 5  stars
I rated Home Alone (1990) 5  stars
I rated Cinema Paradiso (1988) 5  stars
I rated When Harry Met Sally... (1989) 5  stars
I rated Chasing Amy (1997) 4  stars
I rated Shall We Dance? (1996) 5  stars
I rated My Best Friend's Wedding (1997) 4  stars
I rated Deep Rising (1998) 2  stars
I rated Wedding Singer, The (1998) 4  stars
I rated Judge Dredd (1995) 1  stars
I rated Vertigo (1958) 2  stars
I rated True Crime (1995) 2  stars
I rated Rear Window (1954) 2  stars
I rated Dark City (1998) 1  stars


After that, initialize another $R$ matrix based only the movie that I rated already.

In [108]:
myR = np.zeros((len(myMovieRating),1))
for i in range (len(myMovieRating)):
    
    if myMovieRating[i] != 0:
        myR[i] = 1

Y = np.column_stack((myMovieRating,Y))
R = np.column_stack((myR,R))

Now let's get started to build a collaborative filtering algorithm! First, we need to initialize the number of features in each movie (like whether it is drama, crime, romance, SciFi, action, or so on), it can be as much as you want. For the purpose of this project, the number of features = 5 has been chosen.

Then, we also introduce the parameter $X$, which is the feature of each movie. Since the number of features = 5, the $X$ will be a matrix with dimension (total number of movies x 5). We also introduce $\theta$ parameter, which is the user preference in each movie (whether they like a movie with a more action and less drama, for example). Hence, $\theta$ will be a matrix with dimension (total number of users x 5)

To initialize $X$ and $\theta$, it is better if we initialize them randomly.

In [109]:
def initializeWeight(outputVal, inputVal):
    
    epsilon = 2 #It can be adjusted according to your need
    weight = np.zeros((outputVal, inputVal))
    weight = np.random.rand(outputVal, inputVal) * 2*epsilon-epsilon
    
    return weight

totalFeatures = 5

X = initializeWeight(totalMovie, totalFeatures)

# Theta +1 because I add my rating 
Theta = initializeWeight(totalUser+1, totalFeatures) 

Then, we set all of the necessary variables for collaborative filtering algorithm, like regularization parameter, the learning rate, and the number of iterations for gradient descent optimization.

In [110]:
# Set regularization parameter, learning rate, and number of iterations
lambdaReg = 0
alpha = 0.0001
numIterations = 30000

Next would be the complicated algorithm of collaborative filtering. The cost function for collaborative filtering algorithm is showed in the following equation. 

$$J(x^i),...,x^{(n_m)},\theta^{(1)},...,\theta^{(n_u)} = \frac{1}{2} \sum_{(i,j):r(i,j)=1} ((\theta^{(j)})^T x^{(i)}-y^{(i,j)})^2+(\frac{\lambda}{2} \sum_{j=1}^{n^u} \sum_{k=1}^{n} (\theta_k^{(j)})^2)+(\frac{\lambda}{2} \sum_{i=1}^{n_m} \sum_{k=1}^{n} (x_k^{(i)})^2)$$

The cost function is described as the summation of $(\theta*X-y)^2$ where the $R$ matrix = 1, which means that a user has rated the movie. The $\lambda$ expression with double summations in the following term are used for regularization of parameters.

Aside of the cost function $J$, we need to compute the gradient $\theta$, which is the partial derivative of the cost function above with respect to $X$. We also need to compute gradient $X$ simultaneously, which is the partial derivative of the cost function above with respect to $\theta$.


$$ \frac{\partial J}{\partial x_k^{(i)}} = \sum_{j:r(i,j)=1} ((\theta^{(j)})^T x^{(i)}-y^{(i,j)})\theta_k^{(j)}+\lambda x_k^{(i)}$$

$$\frac{\partial J}{\partial \theta_k^{(j)}} = \sum_{i:r(i,j)=1} ((\theta^{(j)})^T x^{(i)}-y^{(i,j)})x_k^{(i)}+\lambda x_k^{(j)}$$

After we compute gradient $\theta$ and $X$, then we can update the value of $\theta$ and $X$ that we previously randomly initialized. Then repeat this process until $n$ number of iterations or until we reach the convergence in the cost function.

In [111]:
def computeCostFunctionCollaborationFiltering (X,Y,theta,lambdaReg, R):
    
    thetaGrad = np.zeros((np.shape(theta)))
    XGrad = np.zeros((np.shape(X)))
    
    costTerm = ((np.matmul(X, theta.transpose()))-Y)**2
    regularizedJ = (((lambdaReg/2))*(np.sum(np.sum(theta**2))))+((lambdaReg/2)*(np.sum(np.sum(X**2))))
    finalJ = ((np.sum(np.sum(np.multiply(costTerm,R))))*0.5) + regularizedJ
    
    XTemp = (np.multiply(((np.matmul(X,theta.transpose()))-Y),R))
    XGrad = (np.matmul(XTemp,theta))+(lambdaReg*X)
    
    thetaTemp = (np.multiply(((np.matmul(X, theta.transpose()))-Y),R))
    thetaGrad = (np.matmul(thetaTemp.transpose(),X))+(lambdaReg*theta)
    
    return finalJ, XGrad, thetaGrad

First, we define a function above to compute the cost function of collaborative filtering algorithm. Then we also define a function to compute gradient descent optimization to learn the best value for both $\theta$ and $X$ parameters.

In [112]:
def computeGradientDescentCollaborationFiltering(x, y, theta, alpha, iterations, lambdaReg, R):
    
    JHistory = np.zeros((iterations,1))
    
    for i in range (iterations):
        
        JHistory[i], xGrad, thetaGrad = computeCostFunctionCollaborationFiltering(x, y, theta, lambdaReg, R)

        x = x - (alpha*xGrad)
        theta = theta - (alpha*thetaGrad)
        
    return JHistory, x, theta

Then, we call the function computeGradientDescentCollaborationFiltering to run the collaborative filtering algorithm. As a return, we get the optimized cost function, as well as the optimized value for both $\theta$ and $X$ parameters.

In [113]:
# Perform optimization process for collaboration filtering algorithm
J, finX, finTheta = computeGradientDescentCollaborationFiltering \
(X, Y, Theta, alpha, numIterations, lambdaReg, R)

Then, we can fetch what movies does the algorithm recommend by multiplying parameter $\theta$ with $X$. Then, sort the result in descending order.

In [114]:
# Fetch my movie recommendation.
moviePredictions = np.matmul(finX,finTheta.transpose())
recommendationForMe = moviePredictions[:,0]

indices = np.argsort(recommendationForMe)[::-1]

Then, we can see what movies the algorithm recommended us. As in the beginning I implied that I like romance, drama, comedy movies considering the rating that I gave, hopefully the algorithm will recommend me movies with the similar genre. 

In [115]:
# Print my movie recommendation.
for i in range (10):
    
    idx = indices[i]
    
    if i == 0:
        print('Top movie recommendations for you:')
    print('No',[i+1],':',movieList[idx])

Top movie recommendations for you:
No [1] : Harlem (1993)
No [2] : Lady of Burlesque (1943)
No [3] : Stonewall (1995)
No [4] : Mixed Nuts (1994)
No [5] : Clean Slate (1994)
No [6] : Country Life (1994)
No [7] : Whole Wide World, The (1996)
No [8] : Drunks (1995)
No [9] : I'm Not Rappaport (1996)
No [10] : Man in the Iron Mask, The (1998)


And there we have it. However, the output can be different for each run since we initialize the $\theta$ and $X$ parameters randomly. Moreover, it is necessary to fine tune the number of iterations, regularization parameter $\lambda$, and the learning rate. Make sure that the number of iteration is enough for the algorithm to converge to give more accurate solutions. In addition, the more we rated the movie, the less the data sparsity will be and the better the algorithm can learn.